<a href="https://colab.research.google.com/github/AnesanR/Pytorch_Tutorials/blob/master/Tutorial_2_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
# Define hpyer-paramerters
input_size=784
num_classes=10
num_epochs=5
batch_size=100
learning_rate=0.001

In [4]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
transform=transforms.ToTensor())

Processing...
Done!


In [0]:
#load datasets
train_loader= torch.utils.data.DataLoader(dataset=train_dataset, batch_size= batch_size, shuffle=True)

test_loader= torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
#create the linear model
model=nn.Linear(input_size, num_classes)

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)  



In [13]:
total_step=len(train_loader)
for epoch in range(num_epochs):
  for i, (images,labels) in enumerate(train_loader):
    #We reshape images to (batch_size, input_size )
    images=images.reshape(-1,28*28)
    
    #Forward pass
    outputs=model(images)
    loss=criterion(outputs,labels)
    
    #Bakpropagate and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (i+1) % 100 == 0:
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
      .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/6], Step [100/600], Loss: 0.9863
Epoch [1/6], Step [200/600], Loss: 0.9810
Epoch [1/6], Step [300/600], Loss: 0.9762
Epoch [1/6], Step [400/600], Loss: 1.0013
Epoch [1/6], Step [500/600], Loss: 0.8609
Epoch [1/6], Step [600/600], Loss: 0.8587
Epoch [2/6], Step [100/600], Loss: 0.9501
Epoch [2/6], Step [200/600], Loss: 0.9617
Epoch [2/6], Step [300/600], Loss: 0.8949
Epoch [2/6], Step [400/600], Loss: 0.8619
Epoch [2/6], Step [500/600], Loss: 0.8438
Epoch [2/6], Step [600/600], Loss: 0.8774
Epoch [3/6], Step [100/600], Loss: 0.9269
Epoch [3/6], Step [200/600], Loss: 0.8686
Epoch [3/6], Step [300/600], Loss: 0.9318
Epoch [3/6], Step [400/600], Loss: 0.7426
Epoch [3/6], Step [500/600], Loss: 0.8930
Epoch [3/6], Step [600/600], Loss: 0.8665
Epoch [4/6], Step [100/600], Loss: 0.7521
Epoch [4/6], Step [200/600], Loss: 0.9133
Epoch [4/6], Step [300/600], Loss: 0.8325
Epoch [4/6], Step [400/600], Loss: 0.7153
Epoch [4/6], Step [500/600], Loss: 0.7967
Epoch [4/6], Step [600/600], Loss:

In [14]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the model on the 10000 test images: 85 %
